In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
with open('fact_dbne_booker.sql') as f:
    lines = f.readlines()
    f.seek(0)
    text = f.read()

### Hundred characters

In [3]:
def hundred_chars(lines):
#     print('Lines of SQL should be no longer than ~100 characters. Please use new line:')
    err_flag = 0
    for count, i in enumerate(lines):
        cnt = 0
        for j in i:
            if j != ' ':
                cnt=cnt+1
        if cnt>100:
            err_flag = 1
            print("\nLine no. {} contains more than 100 characters:\n{}".format(count+1,i.strip()))
    if err_flag == 1:
        print('\n--------------------------------\n')

### Comma at the end of the line

In [4]:
def comma_at_end(lines):
    lines_sign = []
    err_flag = 0
#     print("For following lines comma is present at start of the line: \n")
    for count, i in enumerate(lines):
        if i.lstrip() != '':  
            if i.lstrip()[0].__contains__(","):
                lines_sign.append([count,i])
                err_flag = 1
                print('Line no. {} contains comma at start of the line: \n{}'.format(count,i.strip()))
    if err_flag == 1:
        print('\n--------------------------------\n')

### Inner Join instead of Join keyword 

In [5]:
def inner_join_keyword(lines):
    lines_join = []
    err_flag_inner = 0
    err_flag_on = 0
#     print('Inner keyword not present before join in following lines:\n')
    for count, i in enumerate(lines):
        if i.__contains__("join"):
            lines_join.append([count,i])

    lines_after_join = []
    flag = 0
    for count, i in enumerate(lines):
        if flag == 1:
            flag = 0
            lines_after_join.append([count, i])
        if i.__contains__("join"):
            flag = 1

    join_ls = ['inner', 'left', 'outer', 'right']
    for count, i in enumerate(lines_join):
        flag = 0
        for k in join_ls:
            if flag == 0:
                if k in i[1]:
                    flag = 1
        if flag ==0:
            err_flag_inner = 1
            print("Use Inner keyword on line no. {}\n{}".format(i[0]+1,i[1].strip()))
    if err_flag_inner == 1:
        print('\n--------------------------------\n')
#     print("Use 'On' instead of 'Using' in following lines:\n")
    for count, i in enumerate(lines_after_join):
        if ' using' in i[1]:
            err_flag_on = 1
            print(f"Use 'On' keyword instead of 'Using' on Line no. {i[0]+1}\n{i[1].strip()}")
    if err_flag_on == 1:
        print('\n--------------------------------\n')

### Not equals as !=

In [6]:
def not_equals_sign(lines):
#     print('Use != instead of <> in following lines: \n')
    lines_sign = []
    err_flag = 0
    for count, i in enumerate(lines):
        if i.__contains__("<>"):
            lines_sign.append([count,i])
    for count, i in enumerate(lines_sign):
        err_flag = 1
        print("Use != instead of <> on line no. {}\n{}".format(i[0]+1,i[1].strip()))
    if err_flag == 1:
        print('\n--------------------------------\n')

### Check snake casing

In [7]:
def snake_casing(lines):
    lines_uppercase = []
    flag=0
    err_flag = 0
    reg = '[A-Za-z]+.'
#     print('Following lines contain uppercase characters:\n')
    for count, i in enumerate(lines):
        if i.islower()==False and i!='\n':
            x=i.split("'")
            for cnt,j in enumerate(x):
                if re.search(reg, j):
                    if cnt%2 ==0 and j.islower()==False and j!='\n':
                        flag=1
                        lines_uppercase.append([count+1,i])
                        err_flag = 1
                        print('Line no. {} contains uppercase characters:\n{}'.format(count+1,i.strip()))
    if err_flag == 1:
        print('\n--------------------------------\n')

### CTE PADDING

In [8]:
def cte_padding(lines):
    lines_with_cte = []
    err_flag = 0
    for count, i in enumerate(lines):
        if i.strip() == "with" or i.__contains__("as ("):
            lines_with_cte.append([count,i])
    reg = r'\s*\W\n'
    for i in lines_with_cte:
        if lines[i[0]+1].strip() != '':
            err_flag = 1
            print("Padding absent after line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]-1].strip() != '':
            err_flag = 1
            print("Padding absent before line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]+1].lstrip() == '' and lines[i[0]+2].lstrip() == '':
            err_flag = 1
            print("Extra padding present after line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]-1].strip() == '' and lines[i[0]-2].strip() == '':
            err_flag = 1
            print("Extra padding present before line no {} line - {}".format(i[0]+1,lines[i[0]]))
    lines_select = []
    for count, i in enumerate(lines):
        if i.__contains__("select") and not i.__contains__("select * from"):
            lines_select.append([count,i])
    reg = r'\s*\W\n'
    for i in lines_select:
        if lines[i[0]+1].strip() == '':
            err_flag = 1
            print("Padding present after line no {} line - {}".format(i[0]+1,lines[i[0]]))
        if lines[i[0]-1].strip() == '' and lines[i[0]-2].strip() == '':
            err_flag = 1
            print("Extra padding present before line no {} line - {}".format(i[0]+1,lines[i[0]]))
    if err_flag == 1:
        print('\n--------------------------------\n')

### CTE Alignment

In [9]:
def cte_alignment(text):
    def find_parens(s):
        toret = {}
        pstack = []

        for i, c in enumerate(s):
            if c == '(':
                pstack.append(i)
            elif c == ')':
                toret[pstack.pop()] = i
        return toret
    bracket_dict = find_parens(text)
    opening_bracks = list(bracket_dict.keys())
    closing_bracks = list(bracket_dict.values())
    cte_dict = {}
    err_flag = 0
    for i in bracket_dict.keys():
        if text[i-4:i] == ' as ':
            cte_dict[i] = bracket_dict[i]
    for start_brack in cte_dict.keys():
        k = start_brack
        while text[k] != '\n':
            k=k-1
        cte_name = text[k:start_brack]
        if cte_name[1] == ' ':
            new_line_count = 0
            for i in text[:start_brack]:
                if i=='\n':
                    new_line_count+=1
            err_flag = 1
            print('CTE Alignment incorrect for on line no. {}'.format(new_line_count+1))
        end_brack = cte_dict[start_brack]
        if text[end_brack-1] != '\n':
            new_line_count = 0
            for i in text[:end_brack]:
                if i=='\n':
                    new_line_count+=1
            err_flag = 1
            print('CTE Alignment incorrect for on line no. {}'.format(new_line_count+1))
    if err_flag == 1:
        print('\n--------------------------------\n')


### Select Indentation 

In [10]:
def select_indentation(lines):
    lines_select = []
    err_flag = 0
    for count, i in enumerate(lines):
        if i.__contains__("select") and not i.__contains__("select * from final") and \
        (lines[count-2].__contains__(' as (') or lines[count-1].__contains__('union') or lines[count-1].__contains__('except')):
            lines_select.append([count,i])
    reg = '[a-zA-Z]'
    for j in range(0,len(lines_select)):
        match = re.search(reg,lines_select[j][1])
        if(match.start()!=4):
            err_flag = 1
            print("Select keyword indentation incorrect at line no {}".format(lines_select[j][0]))
    if err_flag == 1:
        print('\n--------------------------------\n')

In [11]:
hundred_chars(lines)
comma_at_end(lines)
inner_join_keyword(lines)
not_equals_sign(lines)
snake_casing(lines)
cte_padding(lines)
cte_alignment(text)
select_indentation(lines)